In [122]:
import os
import requests, json
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor
from langchain_tavily import TavilySearch

In [110]:
load_dotenv("api_key.env", override=True)
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [55]:
@tool
def get_weather(loc):
    """
    retrieve current weather information for a given location using the OpenWeather API \n
    :param loc(str): the name of the location \n
    :return: the JSON-formatted string containing the weather information for the requested location
    """
    url = "https://api.openweathermap.org/data/2.5/weather"

    params = {
        "q": loc,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",
        "lang": "eng_us"
    }

    response = requests.get(url, params=params)

    data = response.json()
    return json.dumps(data)

In [57]:
print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

get_weather
retrieve current weather information for a given location using the OpenWeather API 

:param loc(str): the name of the location 

:return: the JSON-formatted string containing the weather information for the requested location
{'loc': {'title': 'Loc'}}


In [63]:
model = init_chat_model("deepseek-chat", model_provider="deepseek")

In [65]:
tools = [get_weather]
llm_with_tools = model.bind_tools(tools)

In [83]:
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)
get_weather_chain = llm_with_tools | parser | get_weather
get_weather_chain.invoke("Could you please tell me the seattle's weather today?")

'{"coord": {"lon": -122.3321, "lat": 47.6062}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01n"}], "base": "stations", "main": {"temp": 24.44, "feels_like": 24.19, "temp_min": 21.75, "temp_max": 26.74, "pressure": 1013, "humidity": 48, "sea_level": 1013, "grnd_level": 1003}, "visibility": 10000, "wind": {"speed": 0.89, "deg": 180, "gust": 2.68}, "clouds": {"all": 0}, "dt": 1758079708, "sys": {"type": 2, "id": 2041694, "country": "US", "sunrise": 1758030510, "sunset": 1758075589}, "timezone": -25200, "id": 5809844, "name": "Seattle", "cod": 200}'

In [85]:
output_prompt = PromptTemplate.from_template(
    """
    You will get a JSON-formatted weather data, please describe it as a concise, natural way to user. \n
    The JSON-formatted weather data is like: 
    ```json
    {weather_json}
    ```

    please convert it as a natural language weather description in english, return only one sentence.
    """
)

In [87]:
output_chain = output_prompt | model | StrOutputParser()
full_chain = get_weather_chain | output_chain
response = full_chain.invoke("Could you please tell me about seattle's weather today?")

In [89]:
response

"It's a clear night in Seattle with a temperature of 24°C (75°F)."

In [103]:
# create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a assistant, please give relative weather information based on what user asked"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "Could you please tell me about seattle's weather today?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'Seattle'}`
responded: I'll get the current weather information for Seattle for you.

{"coord": {"lon": -122.3321, "lat": 47.6062}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01n"}], "base": "stations", "main": {"temp": 23.55, "feels_like": 23.27, "temp_min": 20.93, "temp_max": 26.03, "pressure": 1013, "humidity": 50, "sea_level": 1013, "grnd_level": 1004}, "visibility": 10000, "wind": {"speed": 1.34, "deg": 236, "gust": 4.02}, "clouds": {"all": 0}, "dt": 1758081076, "sys": {"type": 2, "id": 2046585, "country": "US", "sunrise": 1758030510, "sunset": 1758075589}, "timezone": -25200, "id": 5809844, "name": "Seattle", "cod": 200}Here's the current weather information for Seattle:

**Current Conditions:**
- Clear sky with no clouds
- Temperature: 23.6°F (-4.7°C) - it's quite cold!
- Feels like: 23.3°F (-4.8°C)
- Humidity: 50%
- Wind: Light breeze at 1.34 mph from the southwest


In [105]:
response = agent_executor.invoke({"input": "Could you tell me the weather about Los Angeles and Seattle? Which city is hotter?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'loc': 'Los Angeles'}`
responded: I'll get the current weather information for both Los Angeles and Seattle to compare their temperatures for you.

{"coord": {"lon": -118.2437, "lat": 34.0522}, "weather": [{"id": 800, "main": "Clear", "description": "clear sky", "icon": "01n"}], "base": "stations", "main": {"temp": 25.49, "feels_like": 25.53, "temp_min": 21.25, "temp_max": 27.97, "pressure": 1012, "humidity": 55, "sea_level": 1012, "grnd_level": 993}, "visibility": 10000, "wind": {"speed": 2.57, "deg": 250}, "clouds": {"all": 0}, "dt": 1758081808, "sys": {"type": 2, "id": 2009067, "country": "US", "sunrise": 1758029830, "sunset": 1758074307}, "timezone": -25200, "id": 5368361, "name": "Los Angeles", "cod": 200}
Invoking: `get_weather` with `{'loc': 'Seattle'}`
responded: I'll get the current weather information for both Los Angeles and Seattle to compare their temperatures for you.

{"coord": {"lon": -122.3321, "la

In [126]:
search = TavilySearch(max_results=2)
search.invoke("Upcoming apple products in 2025 September")

{'query': 'Upcoming apple products in 2025 September',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.macrumors.com/guide/upcoming-apple-products/',
   'title': "Upcoming Apple Products Guide: What's Coming in 2025",
   'content': "September 2025\u200b\u200b Apple refreshes the iPhone and Apple Watch lineups each September, and this year we're expecting new iPhone 17 models, an Apple",
   'score': 0.9356824,
   'raw_content': None},
  {'url': 'https://thegadgetflow.com/blog/new-apple-products-coming/',
   'title': '13 New Apple products coming in 2025: What the rumors say',
   'content': 'Looks like the Apple Watch Series 11 will debut on September 9, alongside the iPhone 17 lineup. The timing is perfect—but also dangerous.',
   'score': 0.8849276,
   'raw_content': None}],
 'response_time': 0.95,
 'request_id': '06606ea4-f110-494c-a5ae-cccf7557de6a'}